In [416]:
import dask
import dask.dataframe as ddf
from dask import delayed
import numpy as np
import dask.array as da
import pandas as pd

1) Реализовать алгоритм слияния n(n=2**k) отсортированных списков (произвольной длины) в один отсортированный список

In [407]:
a = [2, 2, 2, 10, 50]
b = [0, 0, 10]
c = [1, 2]
f = [0, 5]
d = [0, 100, 101]

In [412]:
def merge_list(lists):
    
    list_1 = lists.pop(0)
    list_2 = lists.pop(0)
    result = list()
    flag = True
    
    y = list_2.pop(0)
            
    for x in list_1:
        while x > y and flag == True:
            result.append(y)
            if len(list_2) == 0:
                flag = False
                break
            y = list_2.pop(0)
        result.append(x)
        
    
    if y >= result[-1]:
        result.append(y)
        for j in list_2:
            result.append(j)
            
    if len(lists) == 0:
        return result
    else:
        a = lists
        a.insert(0, result)
        return merge_list(a)

In [409]:
merge_list([a, d, f])

[0, 0, 2, 2, 2, 5, 10, 50, 100, 101]

2) С помощью Dask реализовать генерацию n отсортированных списков целых значений (для каждого списка задается собственная длина) и сохранение каждого списка в отдельный файл (в файле хранится 3 столбца: номер_спска, номер_по_порядку, значение. 

In [433]:
def get_array(lenght):
    arr = da.random.randint(0, 100 , size=lenght, chunks=(100))
    return list(np.sort(arr))

def get_n_arrays(n, lenghts):
    output = []
    for i in range(n):
        output.append(get_array(lenght=lenghts[i]))
    return output

In [378]:
def get_table(n, lenghts):
    table = []
    for i in range(n):
        arr = get_array(lenght=lenghts[i])
        for j in range(len(arr)):
            table.append([i, j, arr[j]])
    return table

In [379]:
def save_arr(n, lenghts):
    headers = ['list_id', 'element_id', 'element_value']
    table = []
    for i in range(n):
        arr = get_array(lenght=lenghts[i])
        for j in range(len(arr)):
            table.append([i, j, arr[j]])
        dframe = pd.DataFrame(table, columns=headers)
        dframe.to_csv('arr_{}.csv'.format(i), index = False)
        table = da.array([])

In [376]:
save_arr(3, [5, 7, 8])

In [365]:
get_table(3, [5, 7, 8])

[[0, 0, 0],
 [0, 1, 18],
 [0, 2, 32],
 [0, 3, 35],
 [0, 4, 86],
 [1, 0, 1],
 [1, 1, 1],
 [1, 2, 10],
 [1, 3, 36],
 [1, 4, 37],
 [1, 5, 58],
 [1, 6, 62],
 [2, 0, 37],
 [2, 1, 40],
 [2, 2, 42],
 [2, 3, 45],
 [2, 4, 48],
 [2, 5, 48],
 [2, 6, 84],
 [2, 7, 92]]

3) Применить решение задачи 1 для сортировки данных, сгенерированных в задаче 2

In [385]:
merge_list(get_n_arrays(2, [4, 10]))

[0, 11, 19, 28, 33, 33, 41, 46, 52, 54, 62, 96, 97, 99]

4) При помощи dask.delayed распараллелить решение задачи 1 и применить к данным, сгенерированным в задаче 2

In [431]:
@delayed
def merge_list_4(lists):
    
    list_1 = lists.pop(0)
    list_2 = lists.pop(0)
    result = list()
    flag = True
    
    y = list_2.pop(0)
            
    for x in list_1:
        while x > y and flag == True:
            result.append(y)
            if len(list_2) == 0:
                flag = False
                break
            y = list_2.pop(0)
        result.append(x)
        
    
    if y >= result[-1]:
        result.append(y)
        for j in list_2:
            result.append(j)
            
    return result

In [435]:
z = delayed(merge_list_4)(get_n_arrays(2, [4, 10]))

In [436]:
z.compute()

[2, 10, 21, 53, 68, 72, 75, 75, 78, 83, 85, 87, 89, 97]

In [437]:
n = 4
out = get_n_arrays(n, [4, 10])
results = []
for i in range(n//2):
    y = dask.delayed(merge_list_4)(out)
    results.append(y)
    
result = dask.compute(*results)

In [439]:
result[0]

[3, 4, 11, 15, 29, 30, 41, 44, 46, 55, 60, 61, 82, 84]

5) Модифицировать 4 - обрабатывать данные тем же алгоритмом с помощью pandas.DataFrame

6) C помощью разработанного алгоритма и возможностей Dask.DataFrame отсортировать данные, которые обрабатывались на сминаере по Dask.DataFrame (столбцы: ,id,names,amount,new) по столбцу amount.